In [1]:
import json
from ibm_watson import LanguageTranslatorV3
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.websocket import SynthesizeCallback
from os.path import join, dirname
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Autenticación y credenciales de los servicios en IBM Watson

In [2]:
#ATENTICACIÓN TRANSLATE WATSON
authenticator = IAMAuthenticator('ex7fG5d2ob2SpeqnBatbjrloOjiD3fruLReO-zipIRG8')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator)
language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/3f020a13-ae1f-4d67-87f2-808add2359f5')

#ATENTICACIÓN TEXT_TO_SPEECH WATSON
authenticator = IAMAuthenticator('y6g3Klma5riCxg7RWDp2UUsT9HCW07iaT2qruSdOuTwg')
text_to_speech = TextToSpeechV1( authenticator=authenticator)
text_to_speech.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/6ddfab7b-3699-484e-801b-b2cbcf8cab1c')


#ATENTICACIÓN ASSISTANT WATSON
authenticator = IAMAuthenticator('Vdhmwp71ttlef0bM0JCAVGM1-jqVEt-sQlHqrWFFo_HO')
assistant = AssistantV2(
    version = '2020-04-01',
    authenticator = authenticator
)
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/d80a5eb7-c2a1-43d2-84c0-9891733fc13d')
assistant.set_disable_ssl_verification(False)

session = assistant.create_session('5d54ad00-095a-4d6f-b630-5bb1bb72fea1').get_result()

voices = text_to_speech.list_voices().get_result()
#print(json.dumps(voices, indent=2))

### Identifica el idioma, y traduce de ser el caso

In [ ]:
#CREAR UNA CONDICIÓN PARA QUE LA CONVERSAIÓN SE DETENGA O CONTINUE
condicion=''


# CONVERSASIÓN CON ASSISTANT WATSON
def assistant_bot(pregunta_persona):
    
    message = assistant.message(
        '5d54ad00-095a-4d6f-b630-5bb1bb72fea1',
        session['session_id'],
        input={
            'message_type':'text',
            'text':pregunta_persona
        }
    ).get_result()
    #print(json.dumps(message, indent=2))
    # RECORRO EL JSON RESULT Y OBTENGO EL CAMPO GENERIC QUE ES LA RESPUESTA 
    for key,value in message.items():
        for salida in value["entities"]:
            salida
            
            
        for entityProperty in value["generic"]:
            str(entityProperty)
    
    condicion  = str(salida) 
    res_salida = condicion.split(':',2)
    respuesta = res_salida[1]
    condicion = str(respuesta)
    res_final = condicion.split(',',1)
    condicion =str(res_final[0])
    #print(condicion)
    
    #TEXTO
    response=str(entityProperty)
    res=response.split(':',2)
    paso_valor=str(res[2])
    return paso_valor

#VUELVO A TRADUCIR
def translate_again(texto):
    
    translation = language_translator.translate(
        text = texto, model_id = 'es-en').get_result()
    #print(json.dumps(translation, indent=2, ensure_ascii= False))
    texto=translation['translations']
    transformado=str(texto)
    #print(transformado)
    t=transformado.split(" ", 1)
    #print(t[1])
    v=str(t[1]).split("}",1)
    texto=str(v)
    text_speech_en(texto)      #SE PASA DE TEXTO A VOZ 
    return str(v[0])


# FUNCIÓN DE TRADUCIR 
def translate(texto):
    
    translation = language_translator.translate(
        text = texto, model_id = 'en-es').get_result()
    #print(json.dumps(translation, indent=2, ensure_ascii= False))
    texto=translation['translations']
    transformado=str(texto)
   # print(transformado)
    t=transformado.split(" ", 1)
    #print(t[1])
    v=str(t[1]).split("}",1)
    valor_pasar=str(v[0])
    return valor_pasar

#FUNCIÓN QUE DEVULVE EL TEXTO SIN TRADUCIR
def no_translate(texto):
    
    return texto

#PASAR DE TEXTO A VOZ
def text_speech_en(texto):
    file_path = join( 'resources/output.wav')
    with open(file_path,'wb') as audio_file:
        response = text_to_speech.synthesize(
            texto, accept='audio/wav',
            voice="en-US_AllisonVoice").get_result()
        audio_file.write(response.content)
        
        
        
def text_speech_es(texto):
    file_path = join( 'resources/output.wav')
    with open(file_path,'wb') as audio_file:
        response = text_to_speech.synthesize(
            texto, accept='audio/wav',
            voice="es-LA_SofiaVoice").get_result()
        audio_file.write(response.content)
        
def play_audio():
    pygame.mixer.init()
    pygame.mixer.music.load("resources/output.wav")
    pygame.mixer.music.play()
    pygame.event.wait()



def main():
    print('Escribe algo: ')
    print('Write something: ')
   # print(condicion+" main")
    while str(condicion) != 'despedida':
        texto=input()
        language = language_translator.identify(
            texto).get_result()
        value_language=[]
        for key,value in language.items():
            for lenguage_iden in value:
                values=lenguage_iden.values()
                language_list=list(values)
                value_language.append(language_list[0])
        if value_language[0] == 'en':
            #print(translate(texto))
            pregunta_persona=translate(texto)
            #print(assistant_bot(pregunta_persona))
            try:
                texto=assistant_bot(pregunta_persona)
                print(translate_again(texto))
            except:
                #print(assistant_bot(pregunta_persona))
                print('Quizá la pregunta no está bien formulada')
            
            #
        elif value_language[0] == 'es':
            #print(no_translate(texto))
            pregunta_persona=no_translate(texto)
            
            try:
                texto=assistant_bot(pregunta_persona)
                print(texto)
            except:
                #print(assistant_bot(pregunta_persona))
                print('Quizá la pregunta no está bien formulada')
            text_speech_es(texto)
#play_audio()
main()

Escribe algo: 
Write something: 
good morning
" 'Welcome to the U.P.S. Basic Education career'
How much does the cycle cost?
Quizá la pregunta no está bien formulada
precio de un ciclo


ERROR:root:Invalid Session
Traceback (most recent call last):
  File "D:\Users\leand\anaconda3\envs\IA\lib\site-packages\ibm_cloud_sdk_core\base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Invalid Session, Code: 404 , X-global-transaction-id: c56828b9c98513a628ef6f1e6de8d862


Quizá la pregunta no está bien formulada


## Recibo el texto y traduzco de español a inglés